# Laboratorium 4 - Singular Value Decomposition

##### Aleksandra Mazur

## Zadanie 1 Wyszukiwarka

### 1. Przygotuj duży (> 1000 elementów) zbiór dokumentów tekstowych w języku angielskim (np. wybrany korpus tekstów, podzbiór artykułów Wikipedii, zbiór dokumentów HTML uzyskanych za pomoca Web crawlera, zbiór rozdziałów wyciętych z różnych książek).

Ze strony https://ebible.org/find/details.php?id=eng-web&all=1 pobrano zbiór dokumentów tekstowych w języku angielskim, liczący 1100 plików.

Poniżej przedstawiono kilka przykładowych plików.

In [1]:
import numpy as np
import os
import io

files = os.listdir('documents/')

In [2]:
def show_two_files(files):
    i = 0
    for file in files:
        f = io.open('documents/' + file, encoding="utf8")
        text_from_file = f.read()
        print("File number: ", i)
        print(text_from_file)
        i += 1
        if i == 2:
            break

In [3]:
show_two_files(files)

File number:  0
﻿This set of files contains a script of canonical text, chapter by chapter,
for the purpose of reading to make an audio recording.
All footnotes, introductions, and verse numbers have been stripped out.


File number:  1
﻿The First Book of Moses, Commonly Called Genesis.
Chapter 1.
In the beginning, God created the heavens and the earth. 
The earth was formless and empty. Darkness was on the surface of the deep and God’s Spirit was hovering over the surface of the waters. 
God said, “Let there be light,” and there was light. 
God saw the light, and saw that it was good. God divided the light from the darkness. 
God called the light “day”, and the darkness he called “night”. There was evening and there was morning, the first day. 
God said, “Let there be an expanse in the middle of the waters, and let it divide the waters from the waters.” 
God made the expanse, and divided the waters which were under the expanse from the waters which were above the expanse; and it was s

### 2. Określ słownik słów kluczowych (termów) potrzebny do wyznaczenia wektorów cech bag-of-words (indeksacja). Przykładowo zbiorem takim może być unia wszystkich słów występujących we wszystkich tekstach.

W zadaniu wykorzystano bibliotekę **nltk**, każdą literę we wszystkich słowach zamieniono na małą, usunięto znaki specjalne, kropki, przecinki oraz liczby.

In [4]:
import re
import nltk
nltk.download('punkt')

Słownik słów kluczowych określono jako unię wszystkich słów występujących we wszystkich tekstach.

In [5]:
def create_dictionary(files):
    words_with_quantity = {}
    
    for file in files:
        f = io.open('documents/' + file, encoding="utf8")
        text_from_file = f.read()
        f.close()
        sentences = text_from_file.split('\n')
        
        for sentence in sentences:
            sentence = sentence.lower()
            sentence = re.sub(r'[^\w\s]', '', sentence)
            sentence = re.sub('[0-9]', '', sentence)
            words = nltk.word_tokenize(sentence)
            
            for word in words:
                if word in words_with_quantity.keys():
                    words_with_quantity[word] += 1
                else:
                    words_with_quantity[word] = 1
                    
    return words_with_quantity

In [6]:
words_with_quantity = create_dictionary(files)
dictionary = words_with_quantity.keys()

W *words_with_quantity* zebrano listę wszystkich słów wraz z liczbą wystąpienia każdego z nich.

Poniżej przedstawiono 10 słów, które najczęściej pojawiały się w plikach tekstowych wraz z ilością ich występowania.

In [7]:
def show_most_popular_words(quantity, words_with_quantity):
    words_list = sorted(words_with_quantity.items(), key = lambda x: x[1], reverse=True)
    print("Liczba słów w słowniku: ", len(words_with_quantity))
    print(quantity, " najpopularniejszych słów:")
    for i, element in enumerate(words_list):
        print(element[0], " -> ", element[1])
        if i == quantity - 1:
            break

In [8]:
show_most_popular_words(10, words_with_quantity)

Liczba słów w słowniku:  13675
10  najpopularniejszych słów:
the  ->  53774
of  ->  31534
and  ->  30339
to  ->  19633
you  ->  12157
in  ->  12045
he  ->  9289
will  ->  9088
a  ->  8511
for  ->  8391


### 3. Dla każdego dokumentu j wyznacz wektor cech bag-of-words dj zawierający częstości występowania poszczególnych słów (termów) w tekście.

In [9]:
def create_vector(file, dictionary):
    bow = {}
    for word in dictionary:
        bow[word] = 0
    f = io.open('documents/' + file, encoding="utf8")
    text_from_file = f.read()
    f.close()
    sentences = text_from_file.split('\n')

    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub(r'[^\w\s]', '', sentence)
        sentence = re.sub('[0-9]', '', sentence)
        words = nltk.word_tokenize(sentence)

        for word in words:
            bow[word] += 1
    return bow

In [10]:
def show_words (dictionary, without_zero = True):
    sorted_dictionary = sorted(dictionary.items(), key = lambda x: x[1], reverse=True)
    
    for element in sorted_dictionary:
        if (element[1] == 0 and without_zero):
            break
        print(element[0], " -> ", element[1])

In [11]:
def bag_of_words(quantity, files, dictionary):
    for i, file in enumerate(files):
        print ("\nFile number: ", i)
        bow = create_vector(file, dictionary)
        show_words(bow)

        if i == quantity -1:
            break

Poniżej znajduje się wektor cech bag-of-words dla pierwszego pliku. Przy wyświetlaniu pominięto słowa, które nie występowały w danym tekście.

In [12]:
bag_of_words(1, files, dictionary)


File number:  0
of  ->  3
chapter  ->  2
this  ->  1
set  ->  1
files  ->  1
contains  ->  1
a  ->  1
script  ->  1
canonical  ->  1
text  ->  1
by  ->  1
for  ->  1
the  ->  1
purpose  ->  1
reading  ->  1
to  ->  1
make  ->  1
an  ->  1
audio  ->  1
recording  ->  1
all  ->  1
footnotes  ->  1
introductions  ->  1
and  ->  1
verse  ->  1
numbers  ->  1
have  ->  1
been  ->  1
stripped  ->  1
out  ->  1


### 4. Zbuduj rzadką macierz wektorów cech term-by-document matrix w której wektory cech ułożone są kolumnowo A m×n = [d1|d2| . . . |dn] (m jest liczbą termów w słowniku, a n liczbą dokumentów)

In [13]:
def create_matrix(files, dictionary):
    matrix = []
    for file in files:
        vector = create_vector(file, dictionary)
        matrix.append(vector)
    return matrix

In [14]:
matrix = create_matrix(files, dictionary)

In [15]:
print("Liczba wierszy: ", len(matrix))
print("Liczba kolumn: ", len(matrix[0]))

Liczba wierszy:  1100
Liczba kolumn:  13675


Jak widać liczba wierszy macierzy odpowiada liczbie plików tekstowych, a liczba kolumn zgadza się z ilością słów znajdujących się w zdefiniowanym wcześniej słowniku.

### 5. Przetwórz wstępnie otrzymany zbiór danych mnożąc elementy bag-of-words przez inverse document frequency. Operacja ta pozwoli na redukcje znaczenia często występujących słów. $$IDF(w) = log\frac{N}{n_{w}}$$
#### gdzie $n_{w}$ jest liczbą dokumentów, w których występuje słowo w, a N jest całkowitą liczbą dokumentów.

Poniższa funkcja służy do obliczenia ilości dokumentów, w których występuje dane słowo.

In [16]:
def count_documents_with_word(word):
    counter = 0
    for row in matrix:
        if row.get(word) != 0:
            counter += 1
    return counter

In [17]:
import math

def tf_idf(dictionary):
    for word in dictionary:
        documents_with_word = count_documents_with_word(word)
        if documents_with_word == 0:
            idf = 0
            print(word)
        else:
            idf = float(math.log(len(matrix)/documents_with_word))
        for row in matrix:
            row[word] = row.get(word) * idf

In [18]:
print(count_documents_with_word('chapter'))
print(len(matrix))

1100
1100


Słowo *chapter* występuje w każdym pliku tekstowym, więc po wykonaniu funkcji **tf_idf** jego waga powinna zostać ustawiona na 0.

In [19]:
show_words(matrix[0])
matrix[0].get('chapter')

of  ->  3
chapter  ->  2
this  ->  1
set  ->  1
files  ->  1
contains  ->  1
a  ->  1
script  ->  1
canonical  ->  1
text  ->  1
by  ->  1
for  ->  1
the  ->  1
purpose  ->  1
reading  ->  1
to  ->  1
make  ->  1
an  ->  1
audio  ->  1
recording  ->  1
all  ->  1
footnotes  ->  1
introductions  ->  1
and  ->  1
verse  ->  1
numbers  ->  1
have  ->  1
been  ->  1
stripped  ->  1
out  ->  1


2

In [20]:
tf_idf(dictionary)

In [21]:
show_words(matrix[0])
matrix[0].get('chapter')

files  ->  7.003065458786462
script  ->  7.003065458786462
canonical  ->  7.003065458786462
text  ->  7.003065458786462
audio  ->  7.003065458786462
recording  ->  7.003065458786462
footnotes  ->  7.003065458786462
introductions  ->  7.003065458786462
verse  ->  7.003065458786462
contains  ->  5.9044531701183525
reading  ->  5.616771097666572
stripped  ->  4.112693700890297
purpose  ->  3.6018680771243066
numbers  ->  3.2188758248682006
been  ->  1.3366387706740297
set  ->  0.8982722263714767
make  ->  0.8418581370913855
an  ->  0.7301884522402944
this  ->  0.4378004887511008
by  ->  0.2649129641905048
out  ->  0.23227603487748236
have  ->  0.2208734027796706
all  ->  0.12370965432602289
a  ->  0.04938124791592503
for  ->  0.028586547761416694
to  ->  0.015575211785471372
of  ->  0.0081929955336952
and  ->  0.004555816535860661
the  ->  0.0018198367169858993


0.0

Zgodnie z oczekiwaniami zredukowano do 0 znaczenie słowa, które występowało w każdym tekście. Zatem słowo *chapter* nie jest istotne.

Każda wartość macierzy *matrix* została przemnożona przez *IDF*.

### 6. Napisz program pozwalający na wprowadzenie zapytania (w postaci sekwencji słów) przekształcanego nastpnie do reprezentacji wektorowej q (bag-of-words). Program ma zwrócić k dokumentów najbardziej zbliżonych do podanego zapytania q. Użyj korelacji między wektorami jako miary podobieństwa
$$\cos{\theta_{j}} = \frac{q^{T}d_{j}}{||q|| ||d_{}j||}$$

In [22]:
def create_vector_from_text(text):
    bow = {}
    for word in dictionary:
        bow[word] = 0
        
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub('[0-9]', '', text)
    words = nltk.word_tokenize(text)

    for word in words:
        if word in bow:
            bow[word] += 1
    return list(bow.values())

In [23]:
def get_similar_documents(text, k):
    text_vector = create_vector_from_text(text)
    result = []
    for i, row in enumerate(matrix):
        cos = np.matmul(np.array(text_vector).T, np.array(list(row.values()))) / (np.linalg.norm(text_vector) * np.linalg.norm(list(row.values())))
        
        if (len(result) >= k):
            if cos > result[0][1] or math.isnan(result[0][1]):
                result[0] = (i, cos)
        else:
            result.append((i, cos))
            
        result = sorted(result, key=lambda x: x[1])
    return result

In [24]:
def write_files_at_index(result):
    result = sorted(result, key=lambda x: x[0])
    index = 0
    for i, file in enumerate(files):
        if len(result) == index:
            break
        if i == result[index][0]:
            f = io.open('documents/' + file, encoding="utf8")
            print("\nFile number: ", i)
            print(f.read())
            index += 1  

In [25]:
text = "When Abram was ninety-nine years old, Yahweh appeared to Abram and said to him, “I am God Almighty. Walk before me and be blameless. I will make my covenant between me and you, and will multiply you exceedingly.” Abram fell on his face. God talked with him, saying, ."
result = get_similar_documents(text, 3)

In [26]:
print(result)
write_files_at_index(result)

[(17, 0.28634335213682716), (12, 0.31396209612601805), (15, 0.3461022720388943)]

File number:  12
﻿Genesis.
Chapter 12.
Now Yahweh said to Abram, “Leave your country, and your relatives, and your father’s house, and go to the land that I will show you. 
I will make of you a great nation. I will bless you and make your name great. You will be a blessing. 
I will bless those who bless you, and I will curse him who treats you with contempt. All the families of the earth will be blessed through you.” 
So Abram went, as Yahweh had told him. Lot went with him. Abram was seventy-five years old when he departed from Haran. 
Abram took Sarai his wife, Lot his brother’s son, all their possessions that they had gathered, and the people whom they had acquired in Haran, and they went to go into the land of Canaan. They entered into the land of Canaan. 
Abram passed through the land to the place of Shechem, to the oak of Moreh. At that time, Canaanites were in the land. 
Yahweh appeared to Abram an

Znalezione pliki zawierają według mnie większość słów z wprowadzonego tekstu. Wyszukiwanie zatem działa poprawnie.

### 7. Zastosuj normalizację wektorów cech dj i wektora q, tak aby miały one długość 1. Użyj zmodyfikowanej miary podobieństwa otrzymując
$$ |q^{T}A| = [|\cos{\theta_{1}}|,|\cos{\theta_{2}}|,...,|\cos{\theta_{n}}|]$$

In [27]:
from sklearn.preprocessing import normalize

In [28]:
def normalize_matrix(matrix):
    result = []
    for i, row in enumerate(matrix):
        result.append(normalize([list(row.values())], norm="l1"))
    return result

In [29]:
normalized_matrix = normalize_matrix(matrix)

In [30]:
def get_normalize_vectors(text):
    text_vector_normalized = normalize([create_vector_from_text(text)],norm="l1")
    result = []
    for i, row in enumerate(normalized_matrix):
        cos = np.matmul(np.array(text_vector_normalized[0]).T,np.array(row[0]))/(np.linalg.norm(text_vector_normalized)*np.linalg.norm(row))
        result.append((i, cos))
    return result

In [31]:
def get_similar_documents_normalized(text, k):
    vectors = get_normalize_vectors(text)
    vectors = sorted(vectors, key=lambda x: x[1], reverse=True)
    return vectors[:k]

In [32]:
normalized_result = get_similar_documents_normalized(text, 3)

In [33]:
print(normalized_result)
write_files_at_index(normalized_result)

[(15, 0.3461022720388944), (12, 0.31396209612601805), (17, 0.28634335213682716)]

File number:  12
﻿Genesis.
Chapter 12.
Now Yahweh said to Abram, “Leave your country, and your relatives, and your father’s house, and go to the land that I will show you. 
I will make of you a great nation. I will bless you and make your name great. You will be a blessing. 
I will bless those who bless you, and I will curse him who treats you with contempt. All the families of the earth will be blessed through you.” 
So Abram went, as Yahweh had told him. Lot went with him. Abram was seventy-five years old when he departed from Haran. 
Abram took Sarai his wife, Lot his brother’s son, all their possessions that they had gathered, and the people whom they had acquired in Haran, and they went to go into the land of Canaan. They entered into the land of Canaan. 
Abram passed through the land to the place of Shechem, to the oak of Moreh. At that time, Canaanites were in the land. 
Yahweh appeared to Abram an

Jak widać powyżej znalezione pliki są takie same w obu przypadkach (z normalizacją i bez).

### 8. W celu usunięcia szumu z macierzy A (normalized_matrix) zastosuj SVD i low rank approximation.

Do wykonania tego zadania użyto biblioteki **scipy**.

In [34]:
import scipy

Poniższa funkcja wykorzystuje SVD i low rank approximation.

In [35]:
def svd_and_lra(k):
    A = []
    for row in normalized_matrix:
        A.append(row[0])
    u, s, vt = scipy.sparse.linalg.svds(np.array(A), k=k)
    return u @ np.diag(s) @ vt

Poniższa funkcja zwraca wektor podobieństw danych wejściowych do kolejnych dokumentów.

In [36]:
def get_normalize_vectors_approx(text, k):
    res = svd_and_lra(k)
    text_vector_normalized = normalize([create_vector_from_text(text)],norm="l1")
    result = []
    for i, row in enumerate(res):
        cos = np.matmul(np.array(text_vector_normalized[0]).T,np.array(row))/(np.linalg.norm(text_vector_normalized)*np.linalg.norm(row))
        result.append((i, cos))
    return result

In [37]:
def get_similar_documents_approx(text, k, approx_k):
    vectors = get_normalize_vectors_approx(text, approx_k)
    vectors = sorted(vectors, key=lambda x: x[1], reverse=True)
    return vectors[:k]

In [38]:
approx_result = get_similar_documents_approx(text, 3, 150)

In [39]:
print(approx_result)
write_files_at_index(approx_result)

[(15, 0.3784906427640007), (12, 0.34370772172919767), (13, 0.3315585816066525)]

File number:  12
﻿Genesis.
Chapter 12.
Now Yahweh said to Abram, “Leave your country, and your relatives, and your father’s house, and go to the land that I will show you. 
I will make of you a great nation. I will bless you and make your name great. You will be a blessing. 
I will bless those who bless you, and I will curse him who treats you with contempt. All the families of the earth will be blessed through you.” 
So Abram went, as Yahweh had told him. Lot went with him. Abram was seventy-five years old when he departed from Haran. 
Abram took Sarai his wife, Lot his brother’s son, all their possessions that they had gathered, and the people whom they had acquired in Haran, and they went to go into the land of Canaan. They entered into the land of Canaan. 
Abram passed through the land to the place of Shechem, to the oak of Moreh. At that time, Canaanites were in the land. 
Yahweh appeared to Abram and

Po usunięciu szumu z macierzy i zastosowaniu SVD oraz low rank approximation dla k = 150 otrzymano dwa takie same pliki jak w dwóch wcześniejszych zadaniach, jednak jeden się różnił.

### 9. Porównaj działanie programu bez usuwania szumu i z usuwaniem szumu. Dla jakiej wartości k wyniki wyszukiwania są najlepsze (subiektywnie). Zbadaj wpływ przekształcenia IDF na wyniki wyszukiwania.

In [40]:
print("Bez usuwania szumu:")
print(normalized_result)

Bez usuwania szumu:
[(15, 0.3461022720388944), (12, 0.31396209612601805), (17, 0.28634335213682716)]


In [41]:
for k in [5,10,15,20,25,30,50,100,150,200,300,400,500]:
    print("Dla k = ", k)
    print(get_similar_documents_approx(text, 3, k), "\n")

Dla k =  5
[(836, 0.2960144295143441), (700, 0.2959517503907884), (689, 0.2958445305848699)] 

Dla k =  10
[(700, 0.2972279932137946), (805, 0.295732646363496), (835, 0.29524475026637315)] 

Dla k =  15
[(18, 0.35255787216545026), (891, 0.34634267514283196), (465, 0.3448670466918809)] 

Dla k =  20
[(18, 0.3430897301716176), (722, 0.3411985947798077), (891, 0.33639252068421316)] 

Dla k =  25
[(727, 0.33727809802978564), (722, 0.3335588823105624), (32, 0.3324717323729581)] 

Dla k =  30
[(17, 0.3723578775235692), (15, 0.3657706544410162), (12, 0.348726326995261)] 

Dla k =  50
[(15, 0.3746398588314883), (12, 0.3594304697495081), (13, 0.35186952877661737)] 

Dla k =  100
[(15, 0.37924486169317423), (12, 0.3504441641338397), (13, 0.34450194344269547)] 

Dla k =  150
[(15, 0.3784906427640009), (12, 0.3437077217291979), (13, 0.33155858160665247)] 

Dla k =  200
[(15, 0.38100695392818756), (12, 0.3368981937000188), (13, 0.3215925862419339)] 

Dla k =  300
[(15, 0.3761793511682318), (12, 0.3

Dla **k < 30** program z usuwaniem szumu zwraca pliki wydające się losowymi. Jednak dla k odpowiednio większego **(k >= 30)** wyniki z usuwaniem szumu wydają się być dokładniejsze niż bez jego usuwania. Optymalną wartością według mnie jest **k = 125 (+/- 25)**.

Według mnie **IDF** ma duży wpływ na wyniki wyszukiwania, ponieważ słowa, które występują w większości plików (tak jak wspomniane wcześniej *chapter* lub *a*, *the*, *of*, *and*) nie wpływają aż tak na wynik. Można zatem zwrócić większą uwagę na słowa rzadkie i porównywać pliki pod tym kryterium.